<a href="https://colab.research.google.com/github/FranciscoJSSantos/Analise_dos_dados_criminais_brasil/blob/main/Analise_dos_dados_criminais_brasil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análise de Dados Criminais dos estados do Brasil

O objetivo dessa tarefa é construir um mapa do crime no Brasil a partir de dados abertos. O trabalho pode ser feito em dupla ou individual. Cada dupla ou pessoa deve fazer uma análise e pré-processamento dos dados para que possam construir um mapa do crime do Brasil. Além disso, vocês devem aplicar algoritmos de de clusterização para agrupar cidades de acordo com o nível de criminalidade

In [97]:
# Imports necessários
import pandas as pd
import seaborn as sns
from sklearn import preprocessing
from scipy.spatial import distance_matrix
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
from scipy.spatial.distance import pdist, squareform
from sklearn import metrics

# Carregando bases

In [98]:
# Carregado base ocorrencia
data_ocorrenciasOcorrencia = pd.read_excel("/content/drive/MyDrive/trabalho pratico 2/indicadoressegurancapublicauf.xlsx",sheet_name='Ocorrências', names=['UF', 'Tipo Crime', 'Ano', 'Mês', 'Ocorrências'])
data_ocorrenciasOcorrencia

,UF,Tipo Crime,Ano,Mês,Ocorrências
0,Acre,Estupro,2021,janeiro,39
1,Acre,Furto de veículo,2021,janeiro,55
2,Acre,Homicídio doloso,2021,janeiro,13
3,Acre,Lesão corporal seguida de morte,2021,janeiro,0
4,Acre,Roubo a instituição financeira,2021,janeiro,0
...,...,...,...,...,...
20201,Tocantins,Roubo a instituição financeira,2015,dezembro,6
20202,Tocantins,Roubo de carga,2015,dezembro,1
20203,Tocantins,Roubo de veículo,2015,dezembro,55
20204,Tocantins,Roubo seguido de morte (latrocínio),2015,dezembro,2


In [99]:
# Carregado base vitima
data_ocorrenciasVitima = pd.read_excel("/content/drive/MyDrive/trabalho pratico 2/indicadoressegurancapublicauf.xlsx",sheet_name='Vítimas', names=['UF','Tipo Crime', 'Ano', 'Mês', 'Sexo', 'Vítimas'])
data_ocorrenciasVitima.head(5)

,UF,Tipo Crime,Ano,Mês,Sexo,Vítimas
0,Acre,Homicídio doloso,2021,janeiro,Feminino,1
1,Acre,Homicídio doloso,2021,janeiro,Masculino,12
2,Acre,Homicídio doloso,2021,janeiro,Sexo NI,0
3,Acre,Homicídio doloso,2021,fevereiro,Feminino,4
4,Acre,Homicídio doloso,2021,fevereiro,Masculino,12


Aqui na parte de baixo foi feito um agrupamento com base no estado e UF para ficar melhor destacado o total ocorrencias por tipo no estado:

In [100]:
ocorrencias_por_crime = data_ocorrenciasOcorrencia.groupby(['UF','Tipo Crime']).sum()
ocorrencias_por_crime

Ano  Ocorrências
UF        Tipo Crime                                              
Acre      Estupro                              169512         1494
          Furto de veículo                     169512         2719
          Homicídio doloso                     169512         1962
          Lesão corporal seguida de morte      165474            8
          Roubo a instituição financeira       167493           13
...                                               ...          ...
Tocantins Roubo a instituição financeira       169512           92
          Roubo de carga                       129104           41
          Roubo de veículo                     169512         5125
          Roubo seguido de morte (latrocínio)  169512           93
          Tentativa de homicídio               169512         2768

[243 rows x 2 columns]

Criação de uma variável 'estados' para poder salvar todas as UF numa lista que será usada posteriormente:

In [101]:
# listando os estados
estados = list(data_ocorrenciasOcorrencia.UF.unique())
estados

['Acre',
 'Alagoas',
 'Amapá',
 'Amazonas',
 'Bahia',
 'Ceará',
 'Distrito Federal',
 'Espírito Santo',
 'Goiás',
 'Maranhão',
 'Mato Grosso',
 'Mato Grosso do Sul',
 'Minas Gerais',
 'Pará',
 'Paraíba',
 'Paraná',
 'Pernambuco',
 'Piauí',
 'Rio de Janeiro',
 'Rio Grande do Norte',
 'Rio Grande do Sul',
 'Rondônia',
 'Roraima',
 'Santa Catarina',
 'São Paulo',
 'Sergipe',
 'Tocantins']

Criação de um dataFrame separando todos os tipos de crimes para colunas, fazendo isso podemos ver mais claramente o conjunto e a soma dos dados, também vai ficar melhor de trabalhar mais pra frente

In [102]:
# criando novo DataFrame com a soma das ocorrências de cada tipo de crime
matriz = []

for uf in estados:
  linha = [uf]
  for i in range(9):
    df = ocorrencias_por_crime.query(f"UF == '{uf}'")
    linha.append(df.Ocorrências[i])
  matriz.append(linha)
matriz = pd.DataFrame(matriz, columns=['UF','Estupro', 'Furto_de_veículo',	'Homicídio_doloso'	,'Lesão_corporal_seguida_de_morte'	,'Roubo_a_instituição_financeira',	'Roubo_de_carga',	'Roubo_de_veículo',	'Roubo_seguido_de_morte_(latrocínio)',	'Tentativa_de_homicídio'])
matriz = pd.DataFrame(matriz).set_index('UF')
matriz

,Estupro,Furto_de_veículo,Homicídio_doloso,Lesão_corporal_seguida_de_morte,Roubo_a_instituição_financeira,Roubo_de_carga,Roubo_de_veículo,Roubo_seguido_de_morte_(latrocínio),Tentativa_de_homicídio
UF,,,,,,,,,
Acre,1494,2719,1962,8,13,8,5046,88,704
Alagoas,4088,8287,9515,63,99,429,19193,278,3604
Amapá,2444,4199,1801,117,66,43,2264,135,1489
Amazonas,5902,15961,7679,166,308,78,20609,440,1949
Bahia,19971,35649,37612,591,147,2274,88851,1099,17486
Ceará,11478,31801,24724,219,248,1240,61489,411,8799
Distrito Federal,4449,37217,3087,40,20,314,26944,225,5465
Espírito Santo,4689,30799,8161,95,167,545,28438,278,16670
Goiás,4462,65215,13770,273,300,2526,56321,512,14557


O trecho do código abaixo foi dividido no seguinte:
 - na primeira parte fizemos um delete nas colunas de ano e mês pois para nossa análise não vai servir no momento
 - na segunda parte estamos mesclando os dados da tabela 'Vítimas' com a nova tabela de 'Ocorrências'

In [103]:
del data_ocorrenciasVitima["Ano"]
del data_ocorrenciasVitima["Mês"]

In [104]:
df_vit = data_ocorrenciasVitima.groupby(['UF'])
df_vit = df_vit.agg('sum')

df = matriz.merge(df_vit, how='inner', on='UF')
df

,Estupro,Furto_de_veículo,Homicídio_doloso,Lesão_corporal_seguida_de_morte,Roubo_a_instituição_financeira,Roubo_de_carga,Roubo_de_veículo,Roubo_seguido_de_morte_(latrocínio),Tentativa_de_homicídio,Vítimas
UF,,,,,,,,,,
Acre,1494,2719,1962,8,13,8,5046,88,704,1984
Alagoas,4088,8287,9515,63,99,429,19193,278,3604,10035
Amapá,2444,4199,1801,117,66,43,2264,135,1489,2109
Amazonas,5902,15961,7679,166,308,78,20609,440,1949,8579
Bahia,19971,35649,37612,591,147,2274,88851,1099,17486,41497
Ceará,11478,31801,24724,219,248,1240,61489,411,8799,26641
Distrito Federal,4449,37217,3087,40,20,314,26944,225,5465,3751
Espírito Santo,4689,30799,8161,95,167,545,28438,278,16670,8599
Goiás,4462,65215,13770,273,300,2526,56321,512,14557,14743


In [105]:
columns = ['Estupro',
'Furto_de_veículo',
'Homicídio_doloso',
'Lesão_corporal_seguida_de_morte',
'Roubo_a_instituição_financeira',
'Roubo_de_carga',
'Roubo_de_veículo',
'Roubo_seguido_de_morte_(latrocínio)',
'Tentativa_de_homicídio']

No código abaixo é aplicando uma função de 'normalização dos dados', essa função pega uma dimensão de dados que no caso serião as colunas passadas acima e faria uma transformação desses resultados dentre um intervalo de 0 à 1.

In [106]:
x = df[columns].values 
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(x)
df_norm = pd.DataFrame(x_scaled, index=df.index, columns=columns)
df_norm.head()

,Estupro,Furto_de_veículo,Homicídio_doloso,Lesão_corporal_seguida_de_morte,Roubo_a_instituição_financeira,Roubo_de_carga,Roubo_de_veículo,Roubo_seguido_de_morte_(latrocínio),Tentativa_de_homicídio
UF,,,,,,,,,
Acre,0.000000,0.000000,0.018933,0.000000,0.000000,0.000087,0.007095,0.001712,0.000000
Alagoas,0.034466,0.008555,0.226787,0.050505,0.167969,0.007406,0.043174,0.110160,0.115639
Amapá,0.012622,0.002274,0.014503,0.100092,0.103516,0.000695,0.000000,0.028539,0.031302
Amazonas,0.058568,0.020345,0.176262,0.145087,0.576172,0.001304,0.046785,0.202626,0.049645
Bahia,0.245499,0.050593,1.000000,0.535354,0.261719,0.039479,0.220823,0.578767,0.669192


# Aplicando Algoritmo de Clusterizacao Para Análise

In [107]:
df_dist = pd.DataFrame(
    squareform(pdist(df_norm[columns], metric='euclidean')),
    columns = df.index,
    index = df.index
)

long_form = df_dist.unstack()

long_form.index.rename(["Estado A", "Estado B"], inplace=True)
long_form = long_form.to_frame('distance').reset_index()

long_form[
    (long_form['distance'] < 0.09) 
    & (long_form['Estado A'] != long_form['Estado B'])
]

,Estado A,Estado B,distance
22,Acre,Roraima,0.048049
484,Piauí,Sergipe,0.087835
594,Roraima,Acre,0.048049
692,Sergipe,Piauí,0.087835


In [108]:
for k in range(2, 10):

  kmeans = KMeans(n_clusters=k)
  kmeans.fit(df_norm[columns])
  score_mean = metrics.silhouette_score(df_norm[columns], kmeans.labels_, metric='euclidean')
  print(k, score_mean)

2 0.5004135764779766
3 0.4238726616542479
4 0.3517113399102414
5 0.3383997926668957
6 0.3446303689754932
7 0.3415582750669319
8 0.3449518910831126
9 0.3112649434987426


# Criação do nosso cluster de dados para visualização

In [109]:
kmeans = KMeans(n_clusters=3)
kmeans.fit(df_norm[columns])

KMeans(n_clusters=3)

In [110]:
clusters = kmeans.labels_.tolist()
clusters_dict = { 'UF': list(df.index), 'cluster': clusters}
dataframe_clusters = pd.DataFrame(clusters_dict, columns = ['UF','cluster'])

In [111]:
dataframe_clusters

,UF,cluster
0,Acre,1
1,Alagoas,1
2,Amapá,1
3,Amazonas,1
4,Bahia,0
5,Ceará,0
6,Distrito Federal,1
7,Espírito Santo,1
8,Goiás,0
9,Maranhão,0
